In [2]:
from transformers import AutoTokenizer

# Use the id of the model you want to use
# GPT-2 "openai-community/gpt2"
# Qwen "Qwen/Qwen2-0.5B"
# SmolLM "HuggingFaceTB/SmolLM-135M"

prompt = "It was a dark and stormy day in the"
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-0.5B")
input_ids = tokenizer(prompt).input_ids
input_ids

c:\Users\slope\anaconda3BIS\envs\AIGen\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\slope\anaconda3BIS\envs\AIGen\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\slope\.cache\huggingface\hub\models--Qwen--Qwen2-0.5B. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate deve

[2132, 572, 264, 6319, 323, 13458, 88, 1899, 304, 279]

In [3]:
for t in input_ids:
    print(t, "\t:", tokenizer.decode(t))

2132 	: It
572 	:  was
264 	:  a
6319 	:  dark
323 	:  and
13458 	:  storm
88 	: y
1899 	:  day
304 	:  in
279 	:  the


In [4]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2-0.5B")

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [5]:
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

outputs = model(input_ids)
outputs.logits.shape  # An output for each input token

torch.Size([1, 10, 151936])

In [6]:
final_logits = model(input_ids).logits[0, -1]  # The last set of logits
final_logits.argmax()  # The position of the maximum

tensor(3283)

In [7]:

top10 = torch.topk(final_logits.softmax(dim=0), 10)
for value, index in zip(top10.values, top10.indices):
    print(f"{tokenizer.decode(index):<10} {value.item():.2%}")

NameError: name 'torch' is not defined

In [21]:
output_ids = model.generate(input_ids, max_new_tokens=20)
decoded_text = tokenizer.decode(output_ids[0])

print("Input IDs", input_ids[0])
print("Output IDs", output_ids)
print(f"Generated text: {decoded_text}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


KernelInterrupted: Execution interrupted by the Jupyter kernel.

In [ ]:
beam_output = model.generate(
    input_ids,
    num_beams=5,
    repetition_penalty=2.0,
    max_new_tokens=38,
)

print(tokenizer.decode(beam_output[0]))

In [ ]:
from transformers import set_seed

# Setting the seed ensures we get the same results every time we run this code
set_seed(70)

sampling_output = model.generate(
    input_ids,
    do_sample=True,
    temperature=3.0,
    max_new_tokens=40,
    top_k=0,
)

print(tokenizer.decode(sampling_output[0]))

In [57]:
from transformers import AutoTokenizer

# Use the id of the model you want to use
# GPT-2 "openai-community/gpt2"
# Qwen "Qwen/Qwen2-0.5B"
# SmolLM "HuggingFaceTB/SmolLM-135M"

prompt = "The universe is the fear of many"
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-0.5B")
input_ids = tokenizer(prompt).input_ids
input_ids

[785, 15494, 374, 279, 8679, 315, 1657]

In [59]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2-0.5B")

In [62]:
# We tokenize again but specifying the tokenizer that we want it to
# return a PyTorch tensor, which is what the model expects,
# rather than a list of integers
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

outputs = model(input_ids)
outputs.logits.shape  # An output for each input token

torch.Size([1, 7, 151936])

In [63]:
import torch

# The mistake in the code is that input_ids (of type tensor) is being used directly inside the generate function, 
# but the input to model.generate requires it to be properly formatted for PyTorch.

# Convert the input_ids (already a tensor from prompt tokenization) if needed to the correct format (like explicit batch size)
input_ids_tensor = input_ids  # Assuming input_ids is already a tensor from earlier steps

sampling_output = model.generate(
    input_ids_tensor,
    # Check the token IDs for the words ' positive' and ' negative'
# (note the space before the words)
tokenizer.encode(" positive"), tokenizer.encode(" negative")
    do_sample=True,
    temperature=3.0,
    max_new_tokens=50,
    top_k=50,
)

print(tokenizer.decode(sampling_output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


KeyboardInterrupt: 

Ejemplo paso a paso entendiendo los objetos que intervienen

In [33]:
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
import torch

model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2-0.5B")
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-0.5B")


# Obtener dimensiones
vocab_size = tokenizer.vocab_size
embedding_dim = model.config.hidden_size

print(f"Tamaño del vocabulario: {vocab_size}")
print(f"Dimensiones de los embeddings: {embedding_dim}")


Tamaño del vocabulario: 151643
Dimensiones de los embeddings: 896


In [27]:
#prompt = f"""Question: Is the following review positive or
#negative about the movie?
#Review: {review} Answer:"""

prompt = "A whale is in the ocean"

#Tokenizo el prompt y lo convierto en un tensor de pytorch
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
print(input_ids)



tensor([[   32, 50019,   374,   304,   279, 17951]])


In [29]:
#model(input_ids): Esta línea pasa el tensor input_ids (que contiene los tokens del prompt) a través del modelo de lenguaje.
#outputs: El resultado de esta operación es un objeto que contiene varias salidas del modelo, incluyendo los logits.
outputs = model(input_ids)

#!!!!!!
#Logits: Para generar texto.
#Hidden States:Cada token en la secuencia pasa por múltiples capas del modelo, y en cada capa, se generan representaciones internas
# (hidden states) que capturan la información contextual. Estas representaciones se actualizan en cada capa, integrando información de todos los tokens anteriores.
#Contexto Acumulado: Las representaciones internas de cada token contienen información acumulada del contexto general de la secuencia. Esto permite al modelo entender cómo cada palabra se relaciona con las demás en el contexto.
#Attentions: Para entender la distribución de atención del modelo. 


In [31]:
#logits: Valores sin normalizar que representan la "confianza" del modelo en 
#cada posible palabra siguiente en el vocabulario. Se utilizan para generar texto.
logits = outputs.logits
print(logits)

tensor([[[ 2.7713,  2.5144,  1.7077,  ..., -5.7213, -5.7200, -5.7210],
         [ 3.7050,  4.8398,  1.4650,  ..., -7.0109, -7.0087, -7.0092],
         [ 3.5571,  3.2980, -0.5132,  ..., -6.1721, -6.1712, -6.1717],
         [ 1.5985,  1.3557,  1.1875,  ..., -7.3146, -7.3138, -7.3154],
         [ 3.7240,  0.7333,  0.0122,  ..., -6.0287, -6.0287, -6.0292],
         [ 7.7887,  3.1480,  2.4267,  ..., -7.1898, -7.1884, -7.1901]]],
       grad_fn=<UnsafeViewBackward0>)


In [30]:

last_token_logits = logits[0, -1, :]
print(last_token_logits)

tensor([ 4.8884,  4.8228,  0.7553,  ..., -5.6167, -5.6165, -5.6168],
       grad_fn=<SliceBackward0>)


In [32]:
final_logits = model(input_ids).logits[0, -1]
print(final_logits)

tensor([ 4.8884,  4.8228,  0.7553,  ..., -5.6167, -5.6165, -5.6168],
       grad_fn=<SelectBackward0>)


In [34]:
print(final_logits.softmax(dim=-1))

tensor([1.3354e-05, 1.2505e-05, 2.1411e-07,  ..., 3.6585e-10, 3.6594e-10,
        3.6580e-10], grad_fn=<SoftmaxBackward0>)


In [36]:
import torch

top10 = torch.topk(final_logits.softmax(dim=0), 20)
for value, index in zip(top10.values, top10.indices):
    print(f"{tokenizer.decode(index):<10} {value.item():.2%}")

 I         40.17%
 i         7.59%
 a         6.40%
 my        3.77%
 you       3.26%
 the       1.48%
 it        1.15%
 now       1.03%
 have      0.83%
 here      0.64%
 when      0.60%
 there     0.59%
 love      0.59%
 don       0.58%
 can       0.56%
 this      0.56%
 not       0.53%
 they      0.52%
 want      0.51%
 always    0.49%


In [48]:
sampling_output = model.generate(
    input_ids,
    do_sample=True,
    temperature=2.5,
    max_new_tokens=100,
    top_k=50,
)

print(tokenizer.decode(sampling_output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
I'm a sexy zebra and love to fuck
Bunny and Muppet do fuck stuff again but you will be bored, It says i know if there something or someone new please let us know.. Reply -1 Posted: -59. Posted:: Reply x860 | November | No
Please see "Hot Zephyrs
Fucks me. And my dick. I love cock every night. Come near if i'm a slut or your gonna love what I have
No fucking shame
Posted
I fuck


In [54]:
sampling_output = model.generate(
    input_ids,
    do_sample=True,
    temperature=0.9,
    max_new_tokens=100,
    top_k=40,
)

print(tokenizer.decode(sampling_output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
I'm a sexy zebra and I'm free in the city. A free zebra, I don't have any rules. I'd prefer a sexy friend or a pretty lady. My job is to bring you pleasure but it is up to you to enjoy the show and give them what I want. My free sexy zebra is ready to give you a treat with me. I'm always ready to show you what I can do in the bed. I'm a pussy zebra that wants you to feel like a sex bomb.


Zero-Shot Generalization

In [5]:
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
import torch

model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2-0.5B")
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-0.5B")

/root/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [10]:
# Check the token IDs for the words ' positive' and ' negative'
# (note the space before the words)
tokenizer.encode("positive"), tokenizer.encode("negative")

([30487], [42224])

In [6]:
prompt = "I'm a sexy zebra and I like it."

input_ids = tokenizer(prompt, return_tensors="pt").input_ids
print(input_ids)

tensor([[   40,  2776,   264, 10886,  1147, 50213,   323,   358,  1075,   432,
            13]])


In [8]:
outputs = model(input_ids)
logits = outputs.logits
final_logits = model(input_ids).logits[0, -1]
print(final_logits)

tensor([ 4.5712, -0.8256,  1.7369,  ..., -5.0794, -5.0806, -5.0801],
       grad_fn=<SelectBackward0>)


In [28]:
#cercanía a _positive_
print(final_logits[6785])

tensor(0.8275, grad_fn=<SelectBackward0>)


In [30]:
#cercanía a _negative_
print(final_logits[8225])

tensor(0.2383, grad_fn=<SelectBackward0>)


In [32]:
def score(review):
    """Predict whether it is positive or negative

    This function predicts whether a review is positive or negative
    using a bit of clever prompting. It looks at the logits for the
    tokens ' positive' and ' negative', and returns the label
    with the highest score.
    """
    prompt = f"""Question: Is the following review positive or
negative about the movie?
Review: {review} Answer:"""

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    final_logits = model(input_ids).logits[0, -1]
    if final_logits[6785] > final_logits[8225]:
        print("Positive")
    else:
        print("Negative")

In [36]:
score("It was a so crazy and good time movie..")

Positive


Few-shot Generalization

In [44]:
prompt = """\
Traduce al español:

English: I do not speak Spanish.
Spanish: No hablo español.

English: See you later!
Spanish: ¡Hasta luego!

English: Where is a good restaurant?
Spanish: ¿Dónde hay un buen restaurante?

English: What rooms do you have available?
Spanish: ¿Qué habitaciones tiene disponibles?

English: I like soccer
Spanish:"""
inputs = tokenizer(prompt, return_tensors="pt").input_ids
output = model.generate(
    inputs,
    max_new_tokens=30,
)

print(tokenizer.decode(output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Què te parece este texto?

English: I do not speak Spanish.
Spanish: No hablo español.

English: See you later!
Spanish: ¡Hasta luego!

English: Where is a good restaurant?
Spanish: ¿Dónde hay un buen restaurante?

English: What rooms do you have available?
Spanish: ¿Qué habitaciones tiene disponibles?

English: I like soccer
Spanish: Me gusta el fútbol

English: What is your favorite color?
Spanish: ¿Qué es tu color favorito?

English: What is


Sequence-to-Sequence Tasks, Encoder-decoder models

Between encoder-only, decoder-only, and encoder-decoder models, companies and research labs have released a large number of new open and closed language models, such as GPT-4, Mistral, Falcon, Llama, Qwen, Yi, Claude, Bloom, Gemma, and hundreds more. 

In [20]:
from transformers import pipeline

fill_masker = pipeline("fill-mask", model="bert-base-uncased")
fill_masker("The sexy zebra is [MASK].")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


[{'score': 0.11276628077030182,
  'token': 2908,
  'token_str': 'gone',
  'sequence': 'the sexy zebra is gone.'},
 {'score': 0.062051922082901,
  'token': 2157,
  'token_str': 'right',
  'sequence': 'the sexy zebra is right.'},
 {'score': 0.04646315425634384,
  'token': 2067,
  'token_str': 'back',
  'sequence': 'the sexy zebra is back.'},
 {'score': 0.03310095891356468,
  'token': 2757,
  'token_str': 'dead',
  'sequence': 'the sexy zebra is dead.'},
 {'score': 0.02245509997010231,
  'token': 2045,
  'token_str': 'there',
  'sequence': 'the sexy zebra is there.'}]

<img src="image-20250411-112956.png" width="" align="" />

https://arxiv.org/pdf/2304.04052.pdf

<img src="image-20250411-113659.png" width="" align="" />

Distintos FINE TUNNING:

Token classification
Identify each entity in a sentence, such as a person, location, or organization.

Extractive question answering
Given a paragraph, answer a specific question and extract the answer from the input.

Semantic search
The features generated by the encoder can be handy to build a search system. Given a database of a hundred documents, we can compute the embeddings for each. Then, we can compare the input embeddings with the documents’ embeddings at inference time, hence identifying the most similar document in the database.6

In [22]:
from transformers import pipeline

classifier = pipeline(
    "text-classification",
    model="distilbert/distilbert-base-uncased-finetuned-sst-2-english",
)
classifier("This movie is disgustingly good!")

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Device set to use cpu


[{'label': 'POSITIVE', 'score': 0.9998536109924316}]

some argue that fine-tuning might be unnecessary for most use cases. Instead, prompt engineering could be a more effective and cheaper approach.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=8d9183b8-669b-4ff8-8993-1e340ee425c9' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>